## Gold Layer Processing


In [17]:
# importing libraries
from pyspark.sql.functions import to_timestamp, col, current_timestamp, expr, date_format

StatementMeta(, a08434c9-d66a-4780-90b1-e47c932629b0, 19, Finished, Available, Finished)

In [ ]:
# parameters
processing_timestamp = ""

In [37]:
# reading the nyc_taxi_yellow data (for the latest processed batch) into a dataframe df
df = spark.read.table("silver.nyc_taxi_yellow").filter(f"processing_timestamp = '{processing_timestamp}'")

# reading the lookup data into a dataframe df_pu_lookup
df_pu_lookup = spark.read.table("silver.taxi_zone_lookup")

# reading the lookup data into a dataframe df_do_lookup
df_pu_lookup = spark.read.table("silver.taxi_zone_lookup")

StatementMeta(, a08434c9-d66a-4780-90b1-e47c932629b0, 39, Finished, Available, Finished)

In [38]:
# joining df with df_po_lookup to get pickup information
df = df.join(df_pu_lookup, df["pu_location_id"]==df_pu_lookup["LocationID"], "left")

# joining df with df_do_lookup to get dropoff information        
df = df.join(df_do_lookup, df["do_location_id"]==df_do_lookup["LocationID"], "left")


# selecting only required columns from df
# performing transformations
df = df.\
        select(
                "vendor", 
                # changing to standard date format and aliasing columns
                date_format("tpep_pickup_datetime","yyyy-MM-dd").alias("pickup_date"), 
                date_format("tpep_dropoff_datetime","yyyy-MM-dd").alias("dropoff_date"), 
                df_pu_lookup["Borough"].alias("pickup_borough"), 
                df_do_lookup["Borough"].alias("dropoff_borough"), 
                df_pu_lookup["Zone"].alias("pickup_zone"), 
                df_do_lookup["Zone"].alias("dropoff_zone"), 
                "payment_method", 
                "passenger_count", 
                "trip_distance", 
                "tip_amount", 
                "total_amount",
                "processing_timestamp" )

# writing to gold table
#df.write.mode("append").saveAsTable("gold.nyc_taxi_yellow")

df.write.mode("append").save("Tables/gold/nyc_taxi_yellow")


StatementMeta(, a08434c9-d66a-4780-90b1-e47c932629b0, 40, Finished, Available, Finished)